# User Inputs

In [40]:
#==============
# Load Packages
#==============

# jupyter specifications
from IPython.core.interactiveshell import InteractiveShell
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from simulation import *
pd.options.mode.chained_assignment = None 

#===============
# Settings and User Inputs
#===============

#--------
# Database Login Info
#--------

# postgres login information
pg_log = {
    'USER': 'postgres',
    'PASSWORD': 'Ctdim#1bf!!!!!',
    'HOST': 'localhost',
    'PORT': '5432', 
    'DATABASE_NAME': 'fantasyfootball'
}

# create engine for connecting to database
engine = create_engine('postgres+psycopg2://{}:{}@{}:{}/{}'.format(pg_log['USER'], pg_log['PASSWORD'], pg_log['HOST'],
                                                                   pg_log['PORT'], pg_log['DATABASE_NAME']))

# specify schema and table to write out intermediate results
table_info = {
    'engine': engine,
    'schema': 'websitedev',
}

np.random.seed(123)

#--------
# League Settings
#--------

# set year
year = 2019

# set the number of simulation iterations
iterations = 1000

# define point values for all statistical categories
pass_yd_per_pt = 0.04 
pass_td_pt = 5
int_pts = -2
sacks = -1
rush_yd_per_pt = 0.1 
rec_yd_per_pt = 0.1
rush_rec_td = 7
ppr = 0.5

# creating dictionary containing point values for each position
pts_dict = {}
pts_dict['QB'] = [pass_yd_per_pt, pass_td_pt, rush_yd_per_pt, rush_rec_td, int_pts, sacks]
pts_dict['RB'] = [rush_yd_per_pt, rec_yd_per_pt, ppr, rush_rec_td]
pts_dict['WR'] = [rec_yd_per_pt, ppr, rush_rec_td]
pts_dict['TE'] = [rec_yd_per_pt, ppr, rush_rec_td]

In [41]:
def _bar_center_zero(self, s, color_positive, width):

    # Either the min or the max should reach the edge (50%, centered on zero)
    m = max(abs(s.min()),abs(s.max()))
    normed = s * 60 * width / (100 * m)
    base = 'width: 10em; height: 80%;'

    attrs_pos = (base+ 'background: linear-gradient(90deg, transparent 0%, transparent 5%, {c} 0%, {c} {w}%, '
                'transparent {w}%)')

    return [attrs_pos.format(c=color_positive,  w=(5+x)) for x in normed]

def bar_excel(self, axis=0, color_positive='#5FBA7D', width=100):

    self.apply(self._bar_center_zero, axis=axis, color_positive=color_positive, width=width)
    
    return self

# create the bar charts within the
pd.io.formats.style.Styler._bar_center_zero = _bar_center_zero
pd.io.formats.style.Styler.bar_excel = bar_excel

# Pull in Data

In [42]:
# instantiate simulation class and add salary information to data
sim = FootballSimulation(pts_dict, table_info, 2019, iterations)
d = sim.return_data()

In [60]:
# set league information, included position requirements, number of teams, and salary cap
league_info = {}
league_info['pos_require'] = {'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'FLEX': 2}
league_info['num_teams'] = 12
league_info['initial_cap'] = 295
league_info['salary_cap'] = 295

# input information for players and their associated salaries selected by other teams
to_drop = {}
to_drop['players'] = ['Michael Thomas', 'Damien Williams', 'Mike Evans', 'Robert Woods',
                      'Adam Thielen', 'Ezekiel Elliott', 'Davante Adams', 'Adrian Peterson',
                      'James Conner', 'Aaron Jones', 'Devonta Freeman', 'Kerryon Johnson',
                      'Tyreek Hill', 'Chris Godwin', 'Patrick Mahomes', 'Todd Gurley', 
                      'Christian McCaffrey', 'JuJu Smith-Schuster', 'Saquon Barkley']
to_drop['salaries'] = [55, 16, 65, 19,  
                       46, 114, 71, 11, 
                       11, 23, 11, 30, 
                       36, 11, 11, 90, 
                       77, 43, 110]

# to_drop = {}
# to_drop['players'] = []
# to_drop['salaries'] = []

# input information for players and their associated salaries selected by your team
to_add = {}
to_add['players'] = ['Alvin Kamara', 'Nick Chubb', 'DeAndre Hopkins', 'Brandin Cooks', 
                     'David Johnson', 'Deshaun Watson']
to_add['salaries'] = [35, 18, 45, 57, 96, 11]

results = sim.run_simulation(league_info, to_drop, to_add, iterations=iterations)
avg_sal = sim.show_most_selected(to_add, iterations, num_show=30)
avg_sal.style.bar_excel(color_positive='#5FBA7D')

Remaining Salary: 34.0
Remaining Positions Required: {'QB': 0, 'RB': 0, 'WR': 0, 'TE': 1, 'FLEX': 1}


,Percent Drafted,Average Salary
Trey Burton,10.4,2.5
Austin Hooper,9.6,1.3
Hunter Henry,9.4,15
David Njoku,9,2.5
O.J. Howard,8.9,15.1
Eric Ebron,7.6,2.5
Jimmy Graham,7.6,1.2
Will Dissly,7.2,1.3
Evan Engram,7,19.9
Jared Cook,6.4,6.2
